In [1]:
import pystac_client
import planetary_computer as pc

#import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import shapely
import os
from shapely.geometry import box


In [2]:
def query_line(line, year):

    date_range = str(year)+'-01-01/'+str(year)+'-12-31'

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1")

    search = catalog.search(
        collections=["naip"], 
        intersects=line, 
        datetime=date_range)
    
    items =list(search.get_items()) 
    if len(items)==0:
        return None
    return items

In [3]:
fp = os.path.join(os.getcwd(),'sb_county_coast','sb_county_coast.shp')
coast = gpd.read_file(fp)
coast

,id,geometry
0,0,"LINESTRING (-118.51632 34.07155, -118.80054 34..."


In [4]:
coast_scenes = query_line(coast.geometry[0],2018)

In [5]:
def make_bbox(item):
    c = item.properties['proj:bbox']
    return box(c[0],c[1],c[2],c[3])

In [15]:
# make dataframe with collected scenes and relevant info
df = pd.DataFrame(coast_scenes, columns = ['item'])
df['date']=df.item.apply(lambda item : item.properties['datetime'])
df['crs']=df.item.apply(lambda item : item.properties['proj:epsg'])
df['bbox']=df.item.apply(make_bbox)
df.head(4)

,item,date,crs,bbox
0,<Item id=ca_m_3512060_sw_10_060_20180913_20190...,2018-09-13T00:00:00Z,26910,"POLYGON ((722604 3875466, 722604 3882846, 7164..."
1,<Item id=ca_m_3512060_nw_10_060_20180913_20190...,2018-09-13T00:00:00Z,26910,"POLYGON ((722430 3882402, 722430 3889776, 7162..."
2,<Item id=ca_m_3512052_sw_10_060_20180913_20190...,2018-09-13T00:00:00Z,26910,"POLYGON ((722262 3889332, 722262 3896712, 7160..."
3,<Item id=ca_m_3512052_nw_10_060_20180913_20190...,2018-09-13T00:00:00Z,26910,"POLYGON ((722094 3896268, 722094 3903642, 7159..."


In [11]:
def group_flight(df,date):
    same_date = df[df['date']==date]
    
    crs_list = same_date.crs.unique()
    polygons = []
    for crs in crs_list:
        same_crs = same_date[same_date['crs']==crs]
        area = shapely.ops.unary_union(same_crs.bbox)
        gdf = gpd.GeoDataFrame({'geometry':[area]}, 
                               crs='EPSG:'+str(crs))
        gdf.to_crs('EPSG:4326',inplace=True)
        polygons.append(gdf.geometry[0])

        flight = shapely.ops.unary_union(polygons)

    return flight

In [12]:
def flight_paths(df):
    dates = df.date.unique()
    flights = []
    for date in dates:
        flights.append(group_flight(df,date))
    gdf = gpd.GeoDataFrame({'date':dates, 'geometry':flights},
                     crs = 'EPSG:4326')
    return gdf

In [14]:
flights = flight_paths(df)

In [16]:
flights.to_file(os.path.join(os.getcwd(),'training_set','aux_data','flights.shp'))

/srv/conda/envs/notebook/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
